## Playing with ROUGE

In [1]:
from tensor2tensor.utils import rouge

We can mimic three of the six of Gavrilov's scoring metrics with t2t

### ROUGE-1 and ROUGE-2 F1 score

The general structure here is that each sentence should be a list of words. The method is set up to intake multiple sentences, so the overall input structure is a list of lists. The output is average ROUGE F1 score, so in the following case both sentences have precision and recall of 0.5, so both F1 scores are 0.5, which get averaged to 0.5

In [9]:
generated = [["Hello", "World"], ["My", "name", "is", "Mark"]]
reference = [["Hello", "people"], ["My", "name", "isn't", "Ben"]]
rouge.rouge_n(generated, reference, n=1)

0.5

In [13]:
rouge.rouge_n(generated, reference, n=2)

0.16666666

### ROUGE-L F1 score

In [11]:
rouge.rouge_l_sentence_level(generated, reference)

0.5